In [34]:
from pykrx import stock
import pandas as pd
import mplfinance as mpf
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as ms
import plotly.express as px

In [35]:
# pip install pykrx
# pip install mplfinance

In [36]:
# 전체 종목코드와 종목명 가져오기
stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

In [ ]:
name_list = list(stock_list['종목명'])

dff = pd.DataFrame()
stock_from = "20190104" # 2249~2301
stock_to = "20191206"

for name in name_list:
    stock_name = name
    ticker = stock_list.loc[stock_list['종목명']== stock_name, '종목코드']
    df = stock.get_market_ohlcv_by_date(fromdate=stock_from, todate=stock_to, ticker=ticker)
# 칼럼명을 영문명으로 변경
    df['name'] = name
    df["종가"]=df["종가"].apply(pd.to_numeric,errors="coerce")
    dff = pd.concat([dff,df])
dff = dff.rename(columns={'시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'})

In [ ]:
# 특정 날이 존재하는 종목 리스트 뽑기 

df = dff

dname = df.loc[stock_from]['name']
dname = list(dname.values)
dff = pd.DataFrame()

for name in dname:
    stock_name = name
    ticker = stock_list.loc[stock_list['종목명']== stock_name, '종목코드']
    df = stock.get_market_ohlcv_by_date(fromdate=stock_from, todate=stock_to, ticker=ticker)
    df = df.assign(종목명=stock_name)
# 칼럼명을 영문명으로 변경
    df["종가"]=df["종가"].apply(pd.to_numeric,errors="coerce")
    dff = pd.concat([dff,df])
dff = dff.rename(columns={'시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'})

In [ ]:
# 지표 데이터 프레임 만들기
df = dff

df['ma20'] = df['Close'].rolling(window=20).mean() # 20일 이동평균
df['stddev'] = df['Close'].rolling(window=20).std() # 20일 이동표준편차
df['upper'] = df['ma20'] + 2*df['stddev'] # 상단밴드
df['lower'] = df['ma20'] - 2*df['stddev'] # 하단밴드

df['ma12'] = df['Close'].rolling(window=12).mean() # 12일 이동평균
df['ma26'] = df['Close'].rolling(window=26).mean() # 26일 이동평균
df['MACD'] = df['ma12'] - df['ma26']    # MACD
df['MACD_Signal'] = df['MACD'].rolling(window=9).mean() # MACD Signal(MACD 9일 이동평균)
df['MACD_Oscil'] = df['MACD'] - df['MACD_Signal']   #MACD 오실레이터

df['ndays_high'] = df['High'].rolling(window=14, min_periods=1).max()    # 14일 중 최고가
df['ndays_low'] = df['Low'].rolling(window=14, min_periods=1).min()      # 14일 중 최저가
df['fast_k'] = (df['Close'] - df['ndays_low']) / (df['ndays_high'] - df['ndays_low']) * 100  # Fast %K 구하기
df['slow_d'] = df['fast_k'].rolling(window=3).mean()    # Slow %D 구하기

#RSI 구하기
U = np.where(df['Close'].diff(1) > 0, df['Close'].diff(1), 0)
D = np.where(df['Close'].diff(1) < 0, df['Close'].diff(1) *(-1), 0)
AU = pd.DataFrame(U, index=df.index).rolling(window=14).mean()
AD = pd.DataFrame(D, index=df.index).rolling(window=14).mean()
RSI = AU / (AD+AU) *100
df['RSI'] = RSI

# cci 구하기
df['M'] = 0.0
df['m'] = 0.0
df['D'] = 0.0
df['CCI'] = 0.0

df['M'] = (df['High']+df['Low']+df['Close'])/3

df['m'] = df['M'].rolling(window=20).mean()
df['D'] = (df['M'] - df['m']).rolling(window=20).mean()
df['CCI'] = (df['M']-df['m'])/(0.015*df['D'])

df

In [ ]:
# MFI 구하기

# 여기서 구할수잇나? MFI

len_a = (len(df[df['종목명'] == dname[0]])-38)

df['MF'] = 0
df['MF'] = df['Volume']*df['M']
df['MF_sign'] = 0 

for _ in range(1, len(df)):
    if _%len_a == 0:
        continue
    if df['MF'][_] - df['MF'][_-1] == 0:
        df['MF_sign'][_] = 0
    elif df['MF'][_] - df['MF'][_-1] > 0:
        df['MF_sign'][_] = 1
    elif df['MF'][_] - df['MF'][_-1] < 0:
        df['MF_sign'][_] = -1
        
df

In [ ]:
df['MFI'] = 0

positive = []
negative = []

for _ in range(len(df)-14):
    for i in range(14):
        if df['Volume'][_+i] == 0:
            break
        elif df['MF_sign'][_+i] == -1:
            negative.append(df['MF'][_+i])
        elif df['MF_sign'][_+i] == 1:
            positive.append(df['MF'][_+i])
    if sum(negative) != 0:
        MFR = sum(positive)/sum(negative)
        df['MFI'][_+14] = 100-100/(1+MFR)
    positive = []
    negative = []
df

In [ ]:
# 보조지표 계산에 필요한 NAN값 제거 
df = df.reset_index(drop=False, inplace = False)

a = [0]*38
b = [1]*(len(df[df['종목명'] == dname[0]])-38)
print(b)
c = (a+b)*len(dname)
c = pd.DataFrame(c)

df['bool'] = c
df = df[df['bool'] == 1]

df = df.reset_index(drop=False, inplace = False)
df

In [ ]:
# 거래량 0인적 있는 데이터 제거 

name_list = list(df[df['Volume'] == 0]['종목명'].unique())

# name_list는 volume = 0 인적 잇는것들 
for n in name_list:
    df = df[df['종목명'] != n ]
    
df = df.reset_index(drop=False, inplace = False)
df

In [ ]:
Base_data = df

Base_data['BOL_trade'] = 0 # 밴드의 상한 및 하한선 벗어날 때
Base_data['MACD_trade'] = 0 # 음->양 매수 양->음 매도 
Base_data['RSI_trade'] = 0 # 70이상 매도 , 30이하 매수 
Base_data['CCI_trade'] = 0 # -100 ->상향 매수, 100 -> 하향 매도 
Base_data['STO_trade'] = 0 # k가 d를 상향 돌파, 매수 / k가 d를 하향 돌파 매도 
Base_data['MFI_trade'] = 0 # 20이하로 떨어지면 매수 , 80이상으로 가면 매도 

BOL_c = 0
MACD_c = 0
RSI_c = 0
CCI_c = 0
STO_c = 0
MFI_c = 0


# 볼린저밴드 upper, lower 
# macd = MACD_Oscil
# sto = fast_k, slow_d
# RSI
# CCI

# MFI
Alpha = 0
for i in range(len(Base_data)):
    if Alpha == 0:
        if Base_data['MFI'][i] < 20:
            Alpha = -1
    if Base_data['MFI'][i] > 20:
        if Alpha == -1:
            Base_data['MFI_trade'][i] = 1
            MFI_c += 1
            Alpha = 0
    if Alpha == 0:           
        if Base_data['MFI'][i] > 80:
            Alpha = 1 
    if Base_data['MFI'][i] < 80:
        if Alpha == 1:
            Base_data['MFI_trade'][i] = 2
            Alpha = 0
            MFI_c += 1
            

Alpha = 0 
for i in range(len(Base_data)):
    if Alpha == 0:
        if Base_data['CCI'][i] < -1:
            Alpha = -1 
    if Alpha == -1:
        if Base_data['CCI'][i] > -1:
            Base_data['CCI_trade'][i] = 1
            CCI_c += 1
            Alpha = 0
            
    if Alpha == 0:        
        if Base_data['CCI'][i] > 1:
            Alpha = 1
    if Alpha == 1:
        if Base_data['CCI'][i] < 1:
            Base_data['CCI_trade'][i] = 2
            CCI_c += 1
            Alpha = 0
 
Alpha = 0
for i in range(len(Base_data)):
    if Alpha == 0:
        if Base_data['Close'][i] < Base_data['lower'][i]: # 종가가 하한선보다 작다면 
            Alpha = -1 
    if Base_data['Close'][i] > Base_data['lower'][i]:
        if Alpha == -1:
            Base_data['BOL_trade'][i] = 1
            BOL_c += 1
            Alpha = 0 
            
    if Alpha == 0:
        if Base_data['Close'][i] > Base_data['upper'][i]: # 종가가 하한선보다 크면
            Alpha = 1
    if Base_data['Close'][i] < Base_data['upper'][i]:
        if Alpha == 1:
            Base_data['BOL_trade'][i] = 2
            BOL_c += 1
            Alpha = 0
            
Alpha = 0
for i in range(len(Base_data)):
    if Alpha == 0:
        if Base_data['MACD_Oscil'][i] < 0:
            Alpha = -1
    if Base_data['MACD_Oscil'][i] > 0:
        if Alpha == -1:
            Base_data['MACD_trade'][i] = 1
            MACD_c += 1
            Alpha = 0
            
    if Alpha == 0:           
        if Base_data['MACD_Oscil'][i] > 0:
            Alpha = 1
    if Base_data['MACD_Oscil'][i] < 0:
        if Alpha == 1:
            Base_data['MACD_trade'][i] = 2
            MACD_c += 1
            Alpha = 0
            
Alpha = 0
for i in range(len(Base_data)):
    if Alpha == 0:
        if Base_data['RSI'][i] < 30:
            Alpha = -1
    if Base_data['RSI'][i] > 30:
        if Alpha == -1:
            Base_data['RSI_trade'][i] = 1
            RSI_c += 1
            Alpha = 0
    if Alpha == 0:           
        if Base_data['RSI'][i] > 70:
            Alpha = 1 
    if Base_data['RSI'][i] < 70:
        if Alpha == 1:
            Base_data['RSI_trade'][i] = 2
            RSI_c += 1
            Alpha = 0
    
Alpha = 0
for i in range(len(Base_data)):
    if Alpha == 0:
        if Base_data['fast_k'][i] < Base_data['slow_d'][i]:
            Alpha = -1
    if Alpha == -1:
        if Base_data['fast_k'][i] > Base_data['slow_d'][i]:
            Base_data['STO_trade'][i] = 1
            STO_c += 1
            Alpha = 0
            
    if Alpha == 0:
        if Base_data['fast_k'][i] > Base_data['slow_d'][i]:
            Alpha = 1
    if Alpha == 1:
        if Base_data['fast_k'][i] < Base_data['slow_d'][i]:
            Base_data['STO_trade'][i] = 2
            STO_c += 1
            Alpha = 0


Base_data

In [20]:
# 자본 올인 매매전략
# 매수 발생 시 -> 가능한 양 전량 매수 // 매도 발생 시 가능한 양 전량 매도 

trade_data = Base_data

trade_data['BOL_profit']= 0
trade_data['MACD_profit']= 0
trade_data['RSI_profit']= 0
trade_data['CCI_profit']= 0
trade_data['STO_profit']= 0
trade_data['MFI_profit'] = 0
trade_data['Ori']= 0


# 한 종목 개수 
len_a = len(b)
start_m = 30000000 # 시작 돈 
plus_m = 0 # 월급 

# 0-대기 1-매수 2-매도
def trade(data, asset, indicator):
    day = 0
    ori = 0
    count = 0
    can_b = 0
    can_s = 0
    for _ in range(len(data)-1):
        if _%len_a == 0: # 종목 바뀔때마다 초기화
            day = 0
            ori = 0
            count = 0 
            can_b = 0
            can_s = 0
            asset = start_m
        if day != 0: # 30일마다, plus_m 추가로 들어옴 plus_m이 0이라면 추가금 없는 것 
            if day%30 == 0:
                ori += 1
                day = 0 
                asset += plus_m
                
        if data[indicator + '_trade'][_] == 0: # 아무 신호가 없을 때 
            data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
        elif data[indicator + '_trade'][_] == 1: # 매수 신호 
            can_b = int(asset / data['Open'][_+1]) # 다음날 싯가로 가능한 양 매수 
            asset -= can_b*data['Open'][_+1] 
            count += can_b 
            data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
            can_b = 0
        elif data[indicator + '_trade'][_] == 2: # 매도 신호 
            can_s = int(count) # 현재 보유하고 있는 만큼 판매 가능  
            asset += can_s*data['Open'][_+1] # 다음날 싯가로 판매 
            count -= can_s # count = 0 
            data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
            can_s = 0
        data['Ori'][_] = start_m + (ori)*plus_m
        day += 1
        
    return data

trade_data = trade(trade_data, start_m, "BOL")
trade_data = trade(trade_data, start_m, "MACD")
trade_data = trade(trade_data, start_m, "RSI")
trade_data = trade(trade_data, start_m, "CCI")
trade_data = trade(trade_data, start_m, "STO")
trade_data = trade(trade_data, start_m, "MFI")

trade_data

<ipython-input-20-40a7718071cd>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
<ipython-input-20-40a7718071cd>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Ori'][_] = start_m + (ori)*plus_m
<ipython-input-20-40a7718071cd>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
<ipython-input-20-40a

,level_0,index,날짜,Open,High,Low,Close,Volume,종목명,ma20,...,CCI_bool,STO_bool,MFI_bool,trend,BOL_%,MACD_%,RSI_%,CCI_%,STO_%,MFI_%
0,0,38,2022-03-02,3040,3200,3015,3200,348905,3S,2967.75,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,39,2022-03-03,3200,3245,3175,3245,221081,3S,2993.00,...,1,0,0,1,0.000000,0.000000,0.000000,0.154533,0.000000,0.0
2,2,40,2022-03-04,3235,3275,3180,3250,154415,3S,3009.00,...,1,-1,0,1,0.000000,0.000000,0.000000,0.231800,0.000000,0.0
3,3,41,2022-03-07,3240,3240,3140,3160,111961,3S,3020.25,...,-1,1,0,-1,0.000000,0.000000,0.000000,-1.159000,0.000000,0.0
4,4,42,2022-03-08,3060,3155,3050,3090,87658,3S,3029.75,...,-1,1,0,-1,0.000000,0.000000,0.000000,-2.240733,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197479,216379,314267,2022-06-27,6030,6270,6030,6260,11825,힘스,7186.00,...,1,1,-1,1,-17.086694,-16.001583,-24.016306,-19.681639,-8.738833,0.0
197480,216380,314268,2022-06-28,6160,6290,6160,6290,9326,힘스,7101.50,...,1,1,-1,1,-16.769278,-15.892833,-23.732139,-19.482639,-8.450000,0.0
197481,216381,314269,2022-06-29,6280,6280,6110,6240,7290,힘스,7015.00,...,-1,-1,1,-1,-17.298306,-16.074083,-24.205750,-19.814306,-8.931389,0.0
197482,216382,314270,2022-06-30,6260,6270,6120,6270,4520,힘스,6926.00,...,1,1,-1,1,-16.980889,-15.965333,-23.921583,-19.615306,-8.642556,0.0


In [ ]:
# 분할 매수 매매전략 
# 자본 + 월급 // 매수 발생 시 가능한 양의 50%매수 / 매도 발생 시 가능한 양의 50% 매도 

trade_data = Base_data

trade_data['BOL_profit']= 0
trade_data['MACD_profit']= 0
trade_data['RSI_profit']= 0
trade_data['CCI_profit']= 0
trade_data['STO_profit']= 0

trade_data['MFI_profit'] = 0

trade_data['Ori']= 0


# 한 종목 개수 
len_a = len(b)
start_m = 30000000 # 시작 돈 
plus_m = 3000000 # 월급 

# 0-대기 1-매수 2-매도
def trade(data, asset, indicator):
    day = 0
    ori = 0
    count = 0 
    can_b = 0
    can_s = 0
    for _ in range(len(data)-1):
        if _%len_a == 0:
            day = 0
            ori = 0
            count = 0 
            can_b = 0
            can_s = 0
            asset = start_m
        if day != 0:
            if day%30 == 0:
                ori += 1
                day = 0 
                asset += plus_m
        if data[indicator + '_trade'][_] == 0:
            data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
        elif data[indicator + '_trade'][_] == 1:
            can_b = int(asset / data['Open'][_+1])
            can_b = int(can_b*0.5)
            asset -= can_b*data['Open'][_+1]
            count += can_b 
            data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
            can_b = 0
        elif data[indicator + '_trade'][_] == 2:
            can_s = int(count/2)
            asset += can_s*data['Open'][_+1]
            count -= can_s
            data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
            can_s = 0
        data['Ori'][_] = start_m + (ori)*plus_m
        day += 1
    return data

trade_data = trade(trade_data, start_m, "BOL")
trade_data = trade(trade_data, start_m, "MACD")
trade_data = trade(trade_data, start_m, "RSI")
trade_data = trade(trade_data, start_m, "CCI")
trade_data = trade(trade_data, start_m, "STO")

trade_data = trade(trade_data, start_m, "MFI")

trade_data

In [ ]:
# 맞췄을까? 

trade_data['BOL_bool'] = 0
trade_data['MACD_bool'] = 0
trade_data['RSI_bool'] = 0
trade_data['CCI_bool'] = 0
trade_data['STO_bool'] = 0

trade_data['MFI_bool'] = 0

trade_data['trend'] = 0 

for _ in range(1, len(trade_data)):
    if _%len_a == 0:
        continue
    if trade_data['Close'][_] - trade_data['Close'][_-1] == 0:
        trade_data['trend'][_] = 0
    elif trade_data['Close'][_] - trade_data['Close'][_-1] > 0:
        trade_data['trend'][_] = 1
    elif trade_data['Close'][_] - trade_data['Close'][_-1] < 0:
        trade_data['trend'][_] = -1
        
# trend가 나왔고, 
def gogo(data, indicator):
    state = 0
    for _ in range(len(data)-1):
        if _%len_a == 0:
            state = 0
        # state check
        if data[indicator + '_trade'][_] == 1:
            state = 1 # 매수 상태 
        elif data[indicator + '_trade'][_] == 2:
            state = 2 # 매도 상태
        # state 별로 값 할당 
        if state == 0:
            if data[indicator + '_trade'][_] == 0:
                data[indicator + '_bool'][_] = state
        elif state == 1: # 매수 상태
            # 1 or -1
            if data['trend'][_] >= 0 :
                data[indicator + '_bool'][_] = 1
            else:
                data[indicator + '_bool'][_] = -1
        elif state == 2: # 매도 상태 
            if data['trend'][_] <= 0 :
                data[indicator + '_bool'][_] = 1
            else:
                data[indicator + '_bool'][_] = -1
    return data

gogo(trade_data, 'BOL')
gogo(trade_data, 'MACD')
gogo(trade_data, 'RSI')
gogo(trade_data, 'CCI')
gogo(trade_data, 'STO')
gogo(trade_data, 'MFI')


# 1이 맞춘거, -1이 틀린것 

print(len(trade_data[trade_data['BOL_bool'] == 1]))
print(len(trade_data[trade_data['BOL_bool'] == -1]))

print(len(trade_data[trade_data['MACD_bool'] == 1]))
print(len(trade_data[trade_data['MACD_bool'] == -1]))

print(len(trade_data[trade_data['RSI_bool'] == 1]))
print(len(trade_data[trade_data['RSI_bool'] == -1]))

print(len(trade_data[trade_data['CCI_bool'] == 1]))
print(len(trade_data[trade_data['CCI_bool'] == -1]))

print(len(trade_data[trade_data['STO_bool'] == 1]))
print(len(trade_data[trade_data['STO_bool'] == -1]))

print(len(trade_data[trade_data['MFI_bool'] == 1]))
print(len(trade_data[trade_data['MFI_bool'] == -1]))

# 수익률 %로 환산
trade_data['BOL_%'] = 0
trade_data['MACD_%'] = 0
trade_data['RSI_%'] = 0
trade_data['CCI_%'] = 0
trade_data['STO_%'] = 0

trade_data['MFI_%'] = 0


trade_data['BOL_%'] = ((trade_data['BOL_profit'] - trade_data['Ori'])/trade_data['Ori'])*100
trade_data['MACD_%'] = ((trade_data['MACD_profit'] - trade_data['Ori'])/trade_data['Ori'])*100
trade_data['RSI_%'] = ((trade_data['RSI_profit'] - trade_data['Ori'])/trade_data['Ori'])*100
trade_data['CCI_%'] = ((trade_data['CCI_profit'] - trade_data['Ori'])/trade_data['Ori'])*100
trade_data['STO_%'] = ((trade_data['STO_profit'] - trade_data['Ori'])/trade_data['Ori'])*100

trade_data['MFI_%'] = ((trade_data['MFI_profit'] - trade_data['Ori'])/trade_data['Ori'])*100

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (20,15))
boxplot = trade_data.boxplot(column = ['BOL_%', 'MACD_%','RSI_%','CCI_%','STO_%', 'MFI_%'])
plt.yticks(np.arange(-10, 30, step=3))
plt.show

In [ ]:
# 분할 매매 전략 

print(trade_data['BOL_%'].mean())
print(trade_data['MACD_%'].mean())
print(trade_data['RSI_%'].mean())
print(trade_data['CCI_%'].mean())
print(trade_data['STO_%'].mean())
print(trade_data['MFI_%'].mean())

# stock_from = "20200731" # 2249~2301
# stock_to = "20220701"
# 8.65380358778749
# 13.431442535451176
# 10.337464738962936
# 14.054430406162439
# 13.900234516943659
# 3.152631697118589

In [23]:
# 올인 매매 전략 

print(trade_data['BOL_%'].mean())
print(trade_data['MACD_%'].mean())
print(trade_data['RSI_%'].mean())
print(trade_data['CCI_%'].mean())
print(trade_data['STO_%'].mean())
print(trade_data['MFI_%'].mean())

-1.1463899447041395
0.2699227284373474
-0.3162964505467977
-0.3753219955979413
0.09245516299292258
-0.06083440152654231


In [32]:
trade_data

,level_0,index,날짜,Open,High,Low,Close,Volume,종목명,ma20,...,CCI_bool,STO_bool,MFI_bool,trend,BOL_%,MACD_%,RSI_%,CCI_%,STO_%,MFI_%
0,0,38,2022-03-02,3040,3200,3015,3200,348905,3S,2967.75,...,0,0,0,0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0
1,1,39,2022-03-03,3200,3245,3175,3245,221081,3S,2993.00,...,1,0,0,1,0.000000,0.000000,0.000000,0.30910,0.000000,0.0
2,2,40,2022-03-04,3235,3275,3180,3250,154415,3S,3009.00,...,1,-1,0,1,0.000000,0.000000,0.000000,0.46365,0.000000,0.0
3,3,41,2022-03-07,3240,3240,3140,3160,111961,3S,3020.25,...,-1,1,0,-1,0.000000,0.000000,0.000000,-2.31825,0.000000,0.0
4,4,42,2022-03-08,3060,3155,3050,3090,87658,3S,3029.75,...,-1,1,0,-1,0.000000,0.000000,0.000000,-4.48195,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197479,216379,314267,2022-06-27,6030,6270,6030,6260,11825,힘스,7186.00,...,1,1,-1,1,-28.693767,-24.844867,-31.202133,-30.93250,-4.701333,0.0
197480,216380,314268,2022-06-28,6160,6290,6160,6290,9326,힘스,7101.50,...,1,1,-1,1,-28.352067,-24.844867,-30.872533,-30.60160,-4.244633,0.0
197481,216381,314269,2022-06-29,6280,6280,6110,6240,7290,힘스,7015.00,...,-1,-1,1,-1,-28.921567,-24.844867,-31.421867,-31.15310,-5.005800,0.0
197482,216382,314270,2022-06-30,6260,6270,6120,6270,4520,힘스,6926.00,...,1,1,-1,1,-28.579867,-24.844867,-31.092267,-30.82220,-4.549100,0.0


In [33]:
trade_data = trade_data

,level_0,index,날짜,Open,High,Low,Close,Volume,종목명,ma20,...,CCI_bool,STO_bool,MFI_bool,trend,BOL_%,MACD_%,RSI_%,CCI_%,STO_%,MFI_%
0,0,38,2022-03-02,3040,3200,3015,3200,348905,3S,2967.75,...,0,0,0,0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0
1,1,39,2022-03-03,3200,3245,3175,3245,221081,3S,2993.00,...,1,0,0,1,0.000000,0.000000,0.000000,0.30910,0.000000,0.0
2,2,40,2022-03-04,3235,3275,3180,3250,154415,3S,3009.00,...,1,-1,0,1,0.000000,0.000000,0.000000,0.46365,0.000000,0.0
3,3,41,2022-03-07,3240,3240,3140,3160,111961,3S,3020.25,...,-1,1,0,-1,0.000000,0.000000,0.000000,-2.31825,0.000000,0.0
4,4,42,2022-03-08,3060,3155,3050,3090,87658,3S,3029.75,...,-1,1,0,-1,0.000000,0.000000,0.000000,-4.48195,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197479,216379,314267,2022-06-27,6030,6270,6030,6260,11825,힘스,7186.00,...,1,1,-1,1,-28.693767,-24.844867,-31.202133,-30.93250,-4.701333,0.0
197480,216380,314268,2022-06-28,6160,6290,6160,6290,9326,힘스,7101.50,...,1,1,-1,1,-28.352067,-24.844867,-30.872533,-30.60160,-4.244633,0.0
197481,216381,314269,2022-06-29,6280,6280,6110,6240,7290,힘스,7015.00,...,-1,-1,1,-1,-28.921567,-24.844867,-31.421867,-31.15310,-5.005800,0.0
197482,216382,314270,2022-06-30,6260,6270,6120,6270,4520,힘스,6926.00,...,1,1,-1,1,-28.579867,-24.844867,-31.092267,-30.82220,-4.549100,0.0


In [ ]:
len_a3

In [ ]:
(len(df[df['종목명'] == dname[0]])-38)

In [26]:
MACD_c

14633

In [27]:
BOL_c

12541

In [28]:
STO_c

67317

In [29]:
MFI_c

4352

In [30]:
CCI_c

21996

In [31]:
RSI_c

13299